<a href="https://colab.research.google.com/github/Zanderl1987/QuantConnect-Projects/blob/master/QC_MACD_Covered_Call_Strategy_20200628.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import timedelta

class OptionsAlgorithm(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2014, 11, 1)
        self.SetEndDate(2017, 11, 1)
        self.SetCash(20000)
        self.syl = 'IBM'
        equity = self.AddEquity(self.syl, Resolution.Minute)
        equity.SetDataNormalizationMode(DataNormalizationMode.Raw)
        self.macd = self.MACD(self.syl, 12, 26, 9, MovingAverageType.Exponential, Resolution.Daily)
        self.underlyingsymbol = equity.Symbol
        # use the underlying equity as the benchmark
        self.SetBenchmark(equity.Symbol)
     
        
    def OnData(self,slice):
        
        if self.macd.IsReady:
            if self.Portfolio[self.syl].Quantity == 0 and self.macd.Current.Value > self.macd.Signal.Current.Value:  
                self.Buy(self.syl,100)
            
            # # <1> if there is a MACD short signal, liquidate the stock            
            # elif self.Portfolio[self.syl].Quantity > 0 and self.macd.Current.Value < self.macd.Signal.Current.Value:
            #     self.Liquidate()
            
           
            # # <2> if today's close < lowest close of last 30 days, liquidate the stock   
            # history = self.History([self.syl], 30, Resolution.Daily).loc[self.syl]['close']
            # self.Plot('Stock Plot','stop loss frontier', min(history))
            # if self.Portfolio[self.syl].Quantity > 0:        
            #     if self.Securities[self.syl].Price < min(history):
            #         self.Liquidate()
            
            
            # <3> if there is a MACD short signal, trade the options     
            elif self.Portfolio[self.syl].Quantity > 0 and self.macd.Current.Value < self.macd.Signal.Current.Value:
                try:               
                    if self.Portfolio[self.syl].Invested and not self.Portfolio[self.contract].Invested \
                      and self.Time.hour != 0 and self.Time.minute == 1:
                        self.SellCall() 
                except:
                    if self.Portfolio[self.syl].Invested and self.Time.hour != 0 and self.Time.minute == 1:
                        self.SellCall() 
                        
                
    def BuyPut(self):
        contracts = self.OptionChainProvider.GetOptionContractList(self.underlyingsymbol, self.Time.date())
        if len(contracts) == 0: return
        filtered_contracts = self.InitialFilter(self.underlyingsymbol, contracts, -3, 3, 0, 30)
        put = [x for x in filtered_contracts if x.ID.OptionRight == 1] 
        # sorted the contracts according to their expiration dates and choose the ATM options
        contracts = sorted(sorted(put, key = lambda x: abs(self.Securities[self.syl].Price - x.ID.StrikePrice)), 
                                        key = lambda x: x.ID.Date, reverse=True)
        self.contract = contracts[0]
        self.AddOptionContract(self.contract, Resolution.Minute)
        self.Buy(self.contract, 1)
        
    def SellCall(self):
        contracts = self.OptionChainProvider.GetOptionContractList(self.underlyingsymbol, self.Time.date())
        if len(contracts) == 0: return
        filtered_contracts = self.InitialFilter(self.underlyingsymbol, contracts, -3, 3, 0, 30)
        put = [x for x in filtered_contracts if x.ID.OptionRight == 0] 
        # sorted the contracts according to their expiration dates and choose the ATM options
        contracts = sorted(sorted(put, key = lambda x: abs(self.Securities[self.syl].Price - x.ID.StrikePrice)), 
                                        key = lambda x: x.ID.Date, reverse=True)
        self.contract = contracts[0]
        self.AddOptionContract(self.contract, Resolution.Minute)
        self.Sell(self.contract, 1)

    def InitialFilter(self, underlyingsymbol, symbol_list, min_strike_rank, max_strike_rank, min_expiry, max_expiry):
        
        ''' This method is an initial filter of option contracts
            according to the range of strike price and the expiration date '''
            
        if len(symbol_list) == 0 : return
        # fitler the contracts based on the expiry range
        contract_list = [i for i in symbol_list if min_expiry < (i.ID.Date.date() - self.Time.date()).days < max_expiry]
        # find the strike price of ATM option
        atm_strike = sorted(contract_list,
                            key = lambda x: abs(x.ID.StrikePrice - self.Securities[underlyingsymbol].Price))[0].ID.StrikePrice
        strike_list = sorted(set([i.ID.StrikePrice for i in contract_list]))
        # find the index of ATM strike in the sorted strike list
        atm_strike_rank = strike_list.index(atm_strike)
        try: 
            min_strike = strike_list[atm_strike_rank + min_strike_rank]
            max_strike = strike_list[atm_strike_rank + max_strike_rank]
        except:
            min_strike = strike_list[0]
            max_strike = strike_list[-1]
           
        filtered_contracts = [i for i in contract_list if i.ID.StrikePrice >= min_strike and i.ID.StrikePrice <= max_strike]

        return filtered_contracts